In [19]:
!pip -q install scikit-image pandas matplotlib scipy


In [20]:
from google.colab import drive
drive.mount('/content/drive')

BASE = "/content/drive/MyDrive/petro_project"
IMG_DIR = f"{BASE}/images"
OUT_DIR = f"{BASE}/outputs_fabric"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import matplotlib.pyplot as plt
import numpy as np
import os

def save_fabric_figures(img_rgb, theta, coh, deg, outdir, title):
    os.makedirs(outdir, exist_ok=True)

    # --- Clean mask ---
    coh_mask = coh > 0.15  # softer threshold for visualization

    # --- Coherence map ---
    plt.figure(figsize=(6,6))
    im = plt.imshow(coh, cmap="magma", vmin=0, vmax=1)
    plt.colorbar(im, fraction=0.046, pad=0.04, label="Coherence (anisotropy strength)")
    plt.title(f"{title}\nCoherence Map")
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, "coherence_map.png"), dpi=300)
    plt.close()

    # --- Orientation map (masked) ---
    orient = (np.rad2deg(theta) + 180) % 180
    orient_masked = np.where(coh_mask, orient, np.nan)

    plt.figure(figsize=(6,6))
    im = plt.imshow(orient_masked, cmap="twilight", vmin=0, vmax=180)
    plt.colorbar(im, fraction=0.046, pad=0.04, label="Orientation (°)")
    plt.title(f"{title}\nDominant Fabric Orientation")
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, "orientation_map.png"), dpi=300)
    plt.close()

    # --- Rose plot (normalized) ---
    ang = np.deg2rad(deg % 180.0)
    ang2 = np.concatenate([ang, ang + np.pi])

    bins = 18
    counts, edges = np.histogram(ang2, bins=bins, range=(0, 2*np.pi))
    counts = counts / counts.max()  # normalize for visual comparison
    centers = (edges[:-1] + edges[1:]) / 2

    plt.figure(figsize=(6,6))
    ax = plt.subplot(111, polar=True)
    ax.bar(centers, counts, width=(2*np.pi)/bins, color="steelblue", alpha=0.8)
    ax.set_title(f"{title}\nFabric Rose Diagram", va="bottom")
    ax.set_yticklabels([])
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, "fabric_rose.png"), dpi=300)
    plt.close()


In [31]:
SAMPLE_METADATA = {
    "MYL_01": {"lithology": "mylonite"},
    "MYL_03": {"lithology": "mylonite"},
    "MYL_04": {"lithology": "mylonite"},

    "QVS_01": {"lithology": "quartz_vein_in_schist"},
    "QVS_02": {"lithology": "quartz_vein_in_schist"},
    "QVS_04": {"lithology": "quartz_vein_in_schist"},

    "QZV_01": {"lithology": "quartz_vein"},
    "QZV_02": {"lithology": "quartz_vein"},
}


In [32]:
summ_rows = []

for sid, meta in SAMPLE_METADATA.items():
    sdir = os.path.join(IMG_DIR, sid)
    if not os.path.isdir(sdir):
        print("Missing:", sdir); continue

    files = [f for f in os.listdir(sdir) if f.lower().endswith((".jpg",".jpeg",".png",".tif",".tiff"))]
    if not files:
        print("No image in:", sdir); continue

    for f in files:
        path = os.path.join(sdir, f)
        img = io.imread(path)
        img = crop_circle_roi(img)

        gray = color.rgb2gray(img) if img.ndim==3 else img.astype(np.float32)
        theta, coh = fabric_tensor(gray, sigma=2.5)

        summ = summarize_fabric(theta, coh, coh_min=0.25)
        if summ is None:
            print("Too few coherent pixels:", sid, f); continue

        outdir = os.path.join(OUT_DIR, "per_image", sid)
        save_fabric_figures(img, theta, coh, summ["deg"], outdir, f"{sid} ({meta['lithology']})")

        row = {
            "sample_id": sid,
            "lithology": meta["lithology"],
            "image_file": f,
            "n_pixels_used": summ["n_pixels_used"],
            "coh_mean": summ["coh_mean"],
            "coh_median": summ["coh_median"],
            "coh_frac_high": summ["coh_frac_high"],
            "fabric_R": summ["fabric_R"],
        }
        summ_rows.append(row)

df_summ = pd.DataFrame(summ_rows)
df_summ.to_csv(os.path.join(OUT_DIR, "fabric_image_summaries.csv"), index=False)
df_summ


,sample_id,lithology,image_file,n_pixels_used,coh_mean,coh_median,coh_frac_high,fabric_R
0,MYL_01,mylonite,IMG_4051.JPG,3070986,0.772501,0.828610,0.966015,0.288610
1,MYL_03,mylonite,IMG_4052.JPG,3154249,0.752196,0.801506,0.961743,0.288229
2,MYL_04,mylonite,IMG_4055.JPG,2901329,0.722183,0.763551,0.952810,0.228381
3,QVS_01,quartz_vein_in_schist,IMG_4078.JPG,2938044,0.712387,0.749612,0.947414,0.045350
4,QVS_02,quartz_vein_in_schist,IMG_4069.JPG,2802184,0.742452,0.788749,0.958306,0.101666
5,QVS_04,quartz_vein_in_schist,IMG_4082.JPG,2951577,0.734574,0.778701,0.955029,0.030479
6,QZV_01,quartz_vein,IMG_4427.JPG,2899160,0.702610,0.738742,0.932755,0.087744
7,QZV_02,quartz_vein,IMG_4425.JPG,3006615,0.707357,0.745499,0.940465,0.120247


In [33]:
def save_combined_panel(outdir, title):
    coh_img = plt.imread(os.path.join(outdir, "coherence_map.png"))
    ori_img = plt.imread(os.path.join(outdir, "orientation_map.png"))
    rose_img = plt.imread(os.path.join(outdir, "fabric_rose.png"))

    fig, axs = plt.subplots(1,3, figsize=(15,5))

    axs[0].imshow(coh_img)
    axs[0].set_title("Coherence")
    axs[0].axis("off")

    axs[1].imshow(ori_img)
    axs[1].set_title("Orientation")
    axs[1].axis("off")

    axs[2].imshow(rose_img)
    axs[2].set_title("Rose Plot")
    axs[2].axis("off")

    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, "fabric_panel.png"), dpi=300)
    plt.close()


In [34]:
save_combined_panel(outdir, f"{sid} ({meta['lithology']})")


In [35]:
def boxplot_metric(metric, title, outname):
    groups = ["mylonite","quartz_vein_in_schist","quartz_vein"]
    labels = ["MYL","QVS","QZV"]

    data = [df.loc[df["lithology"]==g, metric].dropna().values for g in groups]

    plt.figure(figsize=(6,5))
    plt.boxplot(data, labels=labels, showfliers=False)
    plt.ylabel(metric)
    plt.title(title)
    plt.grid(axis="y", linestyle="--", alpha=0.4)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, outname), dpi=300)
    plt.close()


In [36]:
import os
import matplotlib.pyplot as plt
import pandas as pd

# Load your summary CSV
df = pd.read_csv(os.path.join(OUT_DIR, "fabric_image_summaries.csv"))

def save_lithology_panel(df, out_path):
    groups = ["mylonite", "quartz_vein_in_schist", "quartz_vein"]
    labels = ["MYL", "QVS", "QZV"]

    metrics = [
        ("coh_mean", "Mean coherence (anisotropy strength)"),
        ("coh_frac_high", "Fraction high-coherence pixels"),
        ("fabric_R", "Preferred orientation strength (R)")
    ]

    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    for ax, (metric, ylabel) in zip(axs, metrics):
        data = [df.loc[df["lithology"]==g, metric].dropna().values for g in groups]
        ax.boxplot(data, labels=labels, showfliers=False)
        ax.set_title(metric)
        ax.set_ylabel(ylabel)
        ax.grid(axis="y", linestyle="--", alpha=0.4)

    fig.suptitle("Fabric anisotropy comparison by lithology", fontsize=14)
    plt.tight_layout()
    plt.savefig(out_path, dpi=300)
    plt.close()

panel_path = os.path.join(OUT_DIR, "lithology_comparison_panel.png")
save_lithology_panel(df, panel_path)
print("Saved:", panel_path)


/tmp/ipython-input-3499092710.py:21: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data, labels=labels, showfliers=False)
/tmp/ipython-input-3499092710.py:21: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data, labels=labels, showfliers=False)
/tmp/ipython-input-3499092710.py:21: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data, labels=labels, showfliers=False)


Saved: /content/drive/MyDrive/petro_project/outputs_fabric/lithology_comparison_panel.png


In [37]:
import numpy as np
import matplotlib.pyplot as plt

def save_scatter_coh_R(df, out_path):
    plt.figure(figsize=(6,5))

    for lith, label in [
        ("mylonite", "MYL"),
        ("quartz_vein_in_schist", "QVS"),
        ("quartz_vein", "QZV"),
    ]:
        sub = df[df["lithology"]==lith]
        plt.scatter(sub["coh_mean"], sub["fabric_R"], label=label)

    plt.xlabel("Mean coherence")
    plt.ylabel("Fabric strength R")
    plt.title("Fabric anisotropy space (coherence vs R)")
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_path, dpi=300)
    plt.close()

scatter_path = os.path.join(OUT_DIR, "coherence_vs_R.png")
save_scatter_coh_R(df, scatter_path)
print("Saved:", scatter_path)


Saved: /content/drive/MyDrive/petro_project/outputs_fabric/coherence_vs_R.png


In [38]:
from PIL import Image

def make_row(samples, out_path, title):
    imgs = []
    for sid in samples:
        base = os.path.join(OUT_DIR, "per_image", sid)
        coh = Image.open(os.path.join(base, "coherence_map.png"))
        ori = Image.open(os.path.join(base, "orientation_map.png"))
        rose = Image.open(os.path.join(base, "fabric_rose.png"))
        imgs.append((coh, ori, rose))

    # stitch each sample as a row of 3, then stack rows
    rows = []
    for coh, ori, rose in imgs:
        w = coh.width + ori.width + rose.width
        h = max(coh.height, ori.height, rose.height)
        row = Image.new("RGB", (w, h), (255,255,255))
        x = 0
        for im in [coh, ori, rose]:
            row.paste(im.convert("RGB"), (x, 0))
            x += im.width
        rows.append(row)

    W = max(r.width for r in rows)
    H = sum(r.height for r in rows)
    canvas = Image.new("RGB", (W, H), (255,255,255))
    y = 0
    for r in rows:
        canvas.paste(r, (0, y))
        y += r.height

    canvas.save(out_path)
    print("Saved:", out_path)

# Example usage:
make_row(["MYL_01"], os.path.join(OUT_DIR, "panel_MYL.png"), "MYL")
make_row(["QVS_01"], os.path.join(OUT_DIR, "panel_QVS.png"), "QVS")
make_row(["QZV_01"], os.path.join(OUT_DIR, "panel_QZV.png"), "QZV")


Saved: /content/drive/MyDrive/petro_project/outputs_fabric/panel_MYL.png
Saved: /content/drive/MyDrive/petro_project/outputs_fabric/panel_QVS.png
Saved: /content/drive/MyDrive/petro_project/outputs_fabric/panel_QZV.png
